<a href="https://colab.research.google.com/github/ketkiambekar/Spam-Classifier/blob/main/Spam_Classifier_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SMS Spam Classifier using Naive Bayes 

In this program, we implement a spam classifier using a very common generative algorithm called Naive Bayes. The Data Corpus used is freely available on Kaggle. Inspite of assuming that word occurences are independent of each other (hence the moniker "Naive"), we get a surprisingly good accuracy of 98% with this classifier.











To begin with, set your messages in the first cell and play all cells. The program will classify your messagess as spam or ham (not spam).  Feel free to add your own messages to this list.

NOTE: Before running this colab, make sure the train, test and val csv files are uploaded to the files section on the left pane. You can find the dataset files on the github page. 

In [ ]:
Messages = []
Messages.append("Urgent! Claim your prize of $2000 now")
Messages.append("Good luck with School, Anette")
Messages.append("Buy our new drug to give you guaranteed results")
Messages.append("Let's go shopping tomorrow!")

In [ ]:
import collections
import numpy as np
import csv

The first step is to clean and normalize our data corpus. We remove punctuations and special characters. 

In [ ]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For simplicity, you should split on whitespace, not
    punctuation or any other character. For normalization, you should convert
    everything to lowercase.  Please do not consider the empty string (" ") to be a word.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """
    #remove punctuations in the message
    punc='~`!@#$%^&*(),.:\";\'-+=_0123456789?£'
    for p in punc:
        message = message.replace(p,'')
    words =  message.lower().split(' ')
    return words
  

We then create a dictionary out of the cleaned list of words. We only select words that occur more than 5 times in all of the Data Corpus. The words are mapped to their indices. 

In [ ]:
def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """
    counts=dict()
    for msg in messages:

        words = get_words(msg)

        #remove duplicates
        words = list(dict.fromkeys(words))

        #count words
        for word in words:
            counts[word] = counts.get(word,0)+1

    my_dict={}
    my_list=[]

    #consider only keys where count >=5
    for key in counts.keys():
        if counts[key]>=5:
            my_list.append(key)

    my_list.sort()    
    for i in range (0,len(my_list)):
        my_dict[my_list[i]]=i
    return my_dict
   


We then create a feature vectors (2D numpy array) for our train messages corresponding to all words in our dictionary, 0 representing absence of word in our message and 1 indicating the presence of the word in our message. 

In [ ]:

def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function creates a numpy array that contains the number of times each word
    of the vocabulary appears in each message. 
    Each row in the resulting array corresponds to each message 
    and each column corresponds to a word of the vocabulary.

    Use the word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
        Where the component (i,j) is the number of occurrences of the
        j-th vocabulary word in the i-th message.
    """
    vocab =  sorted(word_dictionary.keys())
    vocab_length = len(word_dictionary.keys())
    num_messages = len(messages)

    arr = np.zeros((num_messages,vocab_length), dtype=np.float128)
    for i in range(0,num_messages):
        words=get_words(messages[i])
          
        for word in words:
            if word in vocab:
                j = vocab.index(word)
                arr[i][j]+=1

    return arr
  



We then create our model using the Bayesian formula for conditional probability. 

Our model has three parameters: 

phi(j|y1) is the fraction of the spam (y = 1) messages in which a word j does appear.

phi(j|y0) is the fraction of the ham (y = 0) messages in which a word j appears.

phi(j) is overall probability that a message is spam.

We additionally apply Laplace Smoothing, so that words which are not present in our dictionary and therefore exposed to our model for the first time and not automatically given zero weightage. 

In [ ]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    phi_y1_arr = []
    phi_y0_arr = []
    phi_y = 0

    num_messages,vocab = matrix.shape

    #Split into Spam and non spam matrices
    for i in range(0,num_messages):
        if labels[i]==1:
            phi_y1_arr.append(matrix[i])
        else:
            phi_y0_arr.append(matrix[i])
    
    #convert lists to numpy arrays
    phi_y1_np = np.array(phi_y1_arr)
    phi_y0_np = np.array(phi_y0_arr)

    phi_y1 = (1+np.sum(phi_y1_np, axis=0))/(len(phi_y1_np)+vocab)
    phi_y0 = (1+np.sum(phi_y0_np, axis=0))/(len(phi_y0_np)+vocab)

    phi_y= len(phi_y1_np)/num_messages

    return phi_y0, phi_y1, phi_y
    

When Predicting labels, we calculate the maximum log likelihood of a message being spam or ham and compare the two. 

In [ ]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    phi_y0, phi_y1, phi_y = model

    matrix[matrix>1]=1

    y1 = phi_y1*matrix
    y1[y1==0]=1 
    y0 = phi_y0*matrix
    y0[y0==0]=1 
    
    num_messages, vocab = matrix.shape

    p1= np.exp(np.sum(np.log(y1), axis=1))*phi_y
    #denominator = numerator + np.exp(np.sum(np.log(y0), axis=1))*(1-phi_y)

    p0= np.exp(np.sum(np.log(y0), axis=1))*(1-phi_y)
   
    probabilities=[]
    for i in range(0,len(p0)):
        if p0[i]>=p1[i]:
            probabilities.append(0)
        else:
            probabilities.append(1)
    return np.array(probabilities)



Turns out the individual words in and of themselves are indicative of whether a message is spam or not. In the function below, we fetch top 5 words that are most indicatice of a message being spam. 

In [ ]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in part-c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: A list of the top five most indicative words in sorted order with the most indicative first
    """
    #vocab =  sorted(dictionary.keys())

    reversed_dictionary = {v : k for (k, v) in dictionary.items()}
    
    phi_y0,phi_y1,phi_y = model

    logs = np.log(phi_y1)-np.log(phi_y0) 
    sorted_args = np.argsort(logs)
    top_5 = sorted_args[-5:]    
    ret = [reversed_dictionary[i] for i in top_5]

    return ret

In [ ]:
def load_spam_dataset(tsv_path):
    """Load the spam dataset from a TSV file

    Args:
         csv_path: Path to TSV file containing dataset.

    Returns:
        messages: A list of string values containing the text of each message.
        labels: The binary labels (0 or 1) for each message. A 1 indicates spam.
    """

    messages = []
    labels = []

    with open(tsv_path, 'r', newline='', encoding='utf8') as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t', quoting=csv.QUOTE_NONE)

        for label, message in reader:
            messages.append(message)
            labels.append(1 if label == 'spam' else 0)

    return messages, np.array(labels)



Bind it all together in the Main function below. It is also where you can see result of whether your original messages are spam or not. 

In [ ]:
def main():
    train_messages, train_labels = load_spam_dataset('spam_train.tsv')
    val_messages, val_labels = load_spam_dataset('spam_val.tsv')
    test_messages, test_labels = load_spam_dataset('spam_test.tsv')

    dictionary = create_dictionary(train_messages)

    print('Size of dictionary: ', len(dictionary))

  
    train_matrix = transform_text(train_messages, dictionary)

    #np.savetxt('spam_sample_train_matrix', train_matrix[:100,:])

    val_matrix = transform_text(val_messages, dictionary)
    test_matrix = transform_text(test_messages, dictionary)

    naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels)

    naive_bayes_predictions_v = predict_from_naive_bayes_model(naive_bayes_model, val_matrix)

    #np.savetxt('spam_naive_bayes_predictions', naive_bayes_predictions)

    naive_bayes_accuracy_v = np.mean(naive_bayes_predictions_v == val_labels)

    print('\nNaive Bayes had an accuracy of {} on the validation set'.format(naive_bayes_accuracy_v))

    naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)

    naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)

    print('\nNaive Bayes had an accuracy of {} on the testing set'.format(naive_bayes_accuracy))

    top_5_words = get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

    print('\nThe top 5 indicative words for Naive Bayes are: ', top_5_words)


    msg_matrix = transform_text(Messages, dictionary)
    spam_or_ham = predict_from_naive_bayes_model(naive_bayes_model, msg_matrix)

    for i in range(len(Messages)):
      if spam_or_ham[i]==1:
        print("\nThe message \"{}\" is:\nSPAM!".format(Messages[i]))
      else: 
        print("\nThe message \"{}\" is:\nNOT SPAM!".format(Messages[i]))

if __name__ == "__main__":
    main()

Size of dictionary:  1498

Naive Bayes had an accuracy of 0.9820466786355476 on the validation set

Naive Bayes had an accuracy of 0.9874551971326165 on the testing set

The top 5 indicative words for Naive Bayes are:  ['guaranteed', 'tone', 'won', 'prize', 'claim']

The message "Urgent! Claim your prize of $2000 now" is:
SPAM!

The message "Good luck with School, Anette" is:
NOT SPAM!

The message "Buy our new drug to give you guaranteed results" is:
SPAM!

The message "Let's go shopping tomorrow!" is:
NOT SPAM!
